##**Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos**

##### **Integrantes:** Dilan Andrade, Hernán Sánchez y Galo Tarapués

El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

#### **1. Procesamiento del Corpus:**
*   Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).



In [28]:
# Importar las librerías necesarias
import xml.etree.ElementTree as ET

# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}
# Definir una lista de stopwords en español
stopwords = {'de', 'el', 'en', 'la', 'los', 'las', 'y', 'a', 'que', 'para', 'un', 'una', 'es', 'con', 'por', 'del'}


# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    # Filtrar stopwords
    tokens = [token for token in tokens if token not in stopwords]
    return set(tokens)

# Paso 1: Leer y parsear el archivo XML
def parse_corpus(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    corpus = {}
    for doc in root.findall('document'):
        doc_id = int(doc.get('id'))
        title = doc.find('title').text
        keywords = doc.find('keywords').text
        author = doc.find('author').text
        date = doc.find('date').text
        # Procesar las palabras clave
        keyword_set = process_text(keywords)
        corpus[doc_id] = {
            'title': title,
            'keywords': keyword_set,
            'author': author,
            'date': date
        }
    return corpus


In [29]:
corpus = parse_corpus('/content/03ranking_corpus.xml')
corpus

{1: {'title': 'El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.',
  'keywords': {'crónica',
   'médica',
   'salud',
   'tecnología',
   'telemedicina',
   'tratamiento'},
  'author': 'Dr. Juan Pérez',
  'date': '2023-01-15'},
 2: {'title': 'Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.',
  'keywords': {'académico',
   'estudiantes',
   'mental',
   'nutrición',
   'rendimiento',
   'salud'},
  'author': 'Dra. María López',
  'date': '2023-02-10'},
 3: {'title': 'Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.',
  'keywords': {'amistad',
   'bienestar',
   'campus',
   'estudiantil',
   'relaciones',
   'sociales'},
  'author': 'Miguel Rodríguez',
  'date': '2023-03-05'},
 4: {'title': 'El rol de las bibliotecas universitarias en el fomento de la investigación académica.',
  'keywords': {'academia',
   'bibliotecas',
   'investigación',
   'recursos

#### **2. Procesamiento de las Consultas:**
*   Definir las consultas proporcionadas.
*   Extraer las palabras clave de cada consulta.

In [30]:
# Extraer palabras clave de cada consulta usando la función process_text
query_keywords = {query_id: process_text(query) for query_id, query in queries.items()}

# Imprimir las palabras clave de cada consulta
query_keywords

{1: {'académico',
  'estudiantes',
  'impacto',
  'mental',
  'rendimiento',
  'salud',
  'universitarios'},
 2: {'actividades',
  'bienestar',
  'campus',
  'emocional',
  'extracurriculares',
  'universitario'},
 3: {'estrategias', 'estrés', 'estudiantes', 'reducir', 'universitarias'}}

**Generación del Vocabulario**: Recorremos todos los documentos y consultas para extraer el conjunto de todas las palabras clave únicas, construyendo el vocabulario completo.

In [31]:
# Importar la biblioteca numpy para operaciones numéricas
import numpy as np

# Crear un conjunto de todas las palabras clave únicas de los documentos y las consultas
all_keywords = set()

# Agregar palabras clave de cada documento al conjunto all_keywords
for doc in corpus.values():
    all_keywords.update(doc['keywords'])

# Agregar palabras clave de cada consulta al conjunto all_keywords
for query_keywords_set in query_keywords.values():
    all_keywords.update(query_keywords_set)

# Genera un diccionario de palabras clave donde cada palabra clave se asocia a un índice único
keyword_to_index = {keyword: idx for idx, keyword in enumerate(all_keywords)}

# Imprimir el diccionario de palabras clave con sus índices correspondientes
print(keyword_to_index)


{'eventos': 0, 'artificial': 1, 'estudio': 2, 'psicológico': 3, 'redes': 4, 'carga': 5, 'espacios': 6, 'ansiedad': 7, 'bicicletas': 8, 'tendencias': 9, 'tecnologías': 10, 'estrés': 11, 'saludables': 12, 'social': 13, 'exámenes': 14, 'telemedicina': 15, 'mental': 16, 'cognitivo': 17, 'calidad': 18, 'trabajan': 19, 'sostenible': 20, 'investigación': 21, 'salud': 22, 'preventiva': 23, 'servicios': 24, 'estrategias': 25, 'campus': 26, 'sueño': 27, 'hábitos': 28, 'médica': 29, 'adultos': 30, 'comunidades': 31, 'universitario': 32, 'extracurriculares': 33, 'retención': 34, 'grupos': 35, 'laboratorios': 36, 'competencias': 37, 'agotamiento': 38, 'universitaria': 39, 'emocional': 40, 'educación': 41, 'física': 42, 'tratamiento': 43, 'concentración': 44, 'vida': 45, 'cultura': 46, 'verdes': 47, 'equilibrio': 48, 'interacción': 49, 'comparación': 50, 'creatividad': 51, 'ejercicio': 52, 'avances': 53, 'riesgo': 54, 'beneficios': 55, 'recursos': 56, 'ambiental': 57, 'académica': 58, 'desempeño': 5

#### **3. Cálculo de Relevancia:**
*   Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
*   Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

**Funcion de Vectores Binarios y Similitud de Jaccard:** create_binary_vector convierte palabras clave en un vector binario basado en el vocabulario y jaccard_similarity mide la similitud entre dos conjuntos y la division entre su intersección y unión.

In [32]:
# Convierte palabras clave en un vector binario con base en el vocabulario
def create_binary_vector(keywords, keyword_to_index):
    vector = np.zeros(len(keyword_to_index))  # Inicializa un vector de 0s
    for keyword in keywords:
        if keyword in keyword_to_index:       # Si la palabra clave está en el vocabulario
            vector[keyword_to_index[keyword]] = 1  # Asigna 1
    return vector

# Calcula la similitud de Jaccard
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))  # Tamaño de la intersección
    union = len(set1.union(set2))                # Tamaño de la unión
    return intersection / union if union != 0 else 0  # Retorna el valor de Jaccard



**Función para Calcular la Similitud de Coseno:** similitud entre dos vectores calculando el coseno del ángulo entre ellos.

In [33]:
# Función para calcular la similitud de coseno entre dos vectores
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)         # Producto punto entre los dos vectores
    norm_a = np.linalg.norm(vector1)               # magnitud del primer vector
    norm_b = np.linalg.norm(vector2)               # magnitud del segundo vector
    # Retorna la similitud de coseno, manejamos si es que una magnitud es 0 retornamos 0
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0


**Calcular Similitudes entre Consultas y Documentos**: calcula las similitudes entre cada consulta y cada documento utilizando tanto la similitud de coseno como la similitud de Jaccard. Estos se almacenan en un diccionario para cada consulta-documento.

In [34]:
# Creamos un diccionario para almacenar las similitudes
similarities = {query_id: {} for query_id in queries}

# Iterar sobre cada consulta y calcular las similitudes con todos los documentos
for query_id, query_keywords_set in query_keywords.items():
    query_vector = create_binary_vector(query_keywords_set, keyword_to_index)  # Crear el vector binario para la consulta

    for doc_id, doc_info in corpus.items():
        doc_vector = create_binary_vector(doc_info['keywords'], keyword_to_index)  # Crear el vector binario para el documento

        # Calcular similitud de Coseno usando la funcion
        cos_sim = cosine_similarity(query_vector, doc_vector)

        # Calcular similitud de Jaccard usando la funcion
        jaccard_sim = jaccard_similarity(query_keywords_set, doc_info['keywords'])

        # Guardar ambas similitudes en el diccionario
        similarities[query_id][doc_id] = {
            'cosine_similarity': cos_sim,
            'jaccard_similarity': jaccard_sim
        }

# Mostrar las similitudes calculadas
similarities


{1: {1: {'cosine_similarity': 0.1543033499620919,
   'jaccard_similarity': 0.08333333333333333},
  2: {'cosine_similarity': 0.7715167498104595, 'jaccard_similarity': 0.625},
  3: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  4: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  5: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  6: {'cosine_similarity': 0.1543033499620919,
   'jaccard_similarity': 0.08333333333333333},
  7: {'cosine_similarity': 0.7715167498104595, 'jaccard_similarity': 0.625},
  8: {'cosine_similarity': 0.1543033499620919,
   'jaccard_similarity': 0.08333333333333333},
  9: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  10: {'cosine_similarity': 0.0, 'jaccard_similarity': 0.0},
  11: {'cosine_similarity': 0.6172133998483676,
   'jaccard_similarity': 0.4444444444444444},
  12: {'cosine_similarity': 0.33806170189140655, 'jaccard_similarity': 0.2},
  13: {'cosine_similarity': 0.5345224838248487,
   'jaccard_similarity': 0.36363636363636

#### **4. Ranking de Documentos:**
*   Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
*   Mostrar el ranking de documentos para cada consulta.

Ordenamos los documentos para cada consulta según la similitud de cosenoo Jaccard y genera un ranking. Luego, imprimimos el ranking.

In [35]:
# Crear un diccionario para almacenar los rankings
ranked_documents = {query_id: {} for query_id in queries}

# Para cada consulta, ordenar los documentos por la métrica de similitud
for query_id, doc_similarities in similarities.items():
    # Ordenar por similitud de coseno
    sorted_docs = sorted(doc_similarities.items(), key=lambda item: item[1]['cosine_similarity'], reverse=True)

    # El siguiente codigo comentado es si quremos ordenar el codigo mediante la similitud jaccard
    # sorted_docs = sorted(doc_similarities.items(), key=lambda item: item[1]['jaccard_similarity']], reverse=True)

    # Almacenar el ranking de documentos en ranked_documents
    ranked_documents[query_id] = [(doc_id, scores['cosine_similarity'], scores['jaccard_similarity']) for doc_id, scores in sorted_docs]

# Mostrar el ranking de documentos para cada consulta
for query_id, ranking in ranked_documents.items():
    print(f"\nConsulta {query_id}: '{queries[query_id]}'")
    print("Ranking de documentos:")
    for rank, (doc_id, cos_sim, jaccard_sim) in enumerate(ranking, start=1):
        doc_title = corpus[doc_id]['title']  # Obtener el título del documento
        print(f"  {rank}. Documento {doc_id} - Similitud de Coseno: {cos_sim:.4f}, Similitud de Jaccard: {jaccard_sim:.4f}")
        print(f"     Título: {doc_title}")



Consulta 1: 'Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios'
Ranking de documentos:
  1. Documento 2 - Similitud de Coseno: 0.7715, Similitud de Jaccard: 0.6250
     Título: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
  2. Documento 7 - Similitud de Coseno: 0.7715, Similitud de Jaccard: 0.6250
     Título: La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
  3. Documento 14 - Similitud de Coseno: 0.7143, Similitud de Jaccard: 0.5556
     Título: Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
  4. Documento 11 - Similitud de Coseno: 0.6172, Similitud de Jaccard: 0.4444
     Título: Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.
  5. Documento 13 - Similitud de Coseno: 0.5345, Similitud de Jaccard: 0.3636
     Título: Estrategias para